### Search Engine With Tools And Agents

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [ ]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

In [ ]:
tools=[wiki,arxiv]

In [ ]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

In [ ]:
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chat_models import ChatOpenAI
from langchain_classic.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)
rag_chain

In [ ]:
from langchain.tools import tool

@tool
def search_docs(query: str) -> str:
    """
    Search internal knowledge base documents for Langsmith.
    """
    result = rag_chain({"question": query})
    return result["answer"]

In [ ]:
from langchain.agents import create_agent

In [ ]:
agent = create_agent(
    model="gpt-3.5-turbo",   # or "gpt-5" if available
    tools=[search_docs, wiki, arxiv],
    system_prompt="You are a helpful assistant that can search documents and calculate."
)

In [ ]:
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "Tell me about Langsmith"}
    ]
})
# The text answer is usually in

In [ ]:
response['messages'][-1].content

In [ ]:
response['messages'][-2]

In [ ]:
response = agent.invoke({
    "messages": [
        {"content": "What is machine learning?", "role": "user"},
    ]
})

In [ ]:
response['messages'][-1]

In [ ]:
response['messages'][-2]

In [ ]:
response = agent.invoke({
    "messages": [
        {"content": "What's the paper 1706.03762 about?", "role": "user"},
    ]
})

In [ ]:
response["messages"]

In [ ]:
response["messages"][-1].content

In [ ]:
response["messages"][-2]